In [1]:
import sys
import os
from os.path import abspath
import numpy as np
import pandas as pd
from utils.generate_network import generate_network
from utils.prepare_data import prepare_data
from utils.popphy_io import save_params, load_params
from utils.popphy_io import get_stat, get_stat_dict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from models.PopPhy import PopPhyCNN
import warnings
from datetime import datetime
import json
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
#from models.PopPhy2 import ResNet
from models.PopPhy2 import ResNet

# Data Preparation

### Reading Configuration
Configuring which data to read in, minimun threshold needed in an OTU (individual sample must have at least set threshold relative abundance), and how many k folds for k fold cross validation.

In [9]:
dataset = 'skg-wt-t1'
threshold = 0
k = 5

### Reduce Features
Reduce amount of OTU features by filtering out OTUs that contain no individual sample with a relative abundance greater than the set threshold.

In [3]:
path = "../data/" + dataset
data = pd.read_csv(path + '/abundance.tsv', index_col=0, sep='\t', header=None)
to_drop = data.loc[(data < threshold).all(axis=1)]
data = data.drop(to_drop.index)

data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,,,,,,,,,,,,,,,,,,
k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__Bacteroidales|f__Bacteroidaceae|g__Bacteroides|s__Bacteroides_acidifaciens,35,712,107,0,616,609,1129,397,209,562,284,1923,989,815,1254,2941,1171,716
k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__unclassified|s__unclassified_unclassified,0,0,9,0,0,0,0,0,0,0,0,355,0,2986,0,4,2,0
k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus|s__Lactobacillus_murinus,294,258,318,258,403,446,914,761,6,1150,664,4588,1151,1660,3936,920,721,509
k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus|s__Lactobacillus_johnsonii,15,135,92,470,125,54,30,26,52,1761,3105,866,679,430,2288,1161,2464,1215
k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipes|s__Alistipes_unclassified,46,106,12,373,52,73,98,0,13,971,518,181,130,998,777,92,28,185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
k__Bacteria|p__Patescibacteria|c__Saccharimonadia|o__Saccharimonadales|f__Saccharimonadaceae|g__Candidatus Saccharimonas|s__Candidatus Saccharimonas_unclassified,0,0,0,3,0,0,0,0,0,0,0,0,0,2,0,0,0,0
k__Bacteria|p__Firmicutes|c__Clostridia|o__Lachnospirales|f__Lachnospiraceae|g__GCA-900066575|s__GCA-900066575_unclassified,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridia UCG-014|f__unclassified|g__unclassified|s__unclassified_unclassified,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0


### Create 2d Matrix Representing OTU Data
Dai et al. PopPhy-CNN's (2019) algorithm creates Phylogenetic tree from OTUs and populates tree based on OTU abundances. This tree graph structure is then converted to a 2d Matrix by taking each parent node in the tree graph and pushing them all to the left and childrens' nodes in the same order from left to right the parents were ordered.

In [4]:
my_maps, raw_x, tree_x, raw_features, tree_features, labels, label_set, g, feature_df = prepare_data(path, data)

# norms = np.linalg.norm(my_maps, axis=2, keepdims=True)
# my_maps = my_maps / norms
pd.DataFrame(my_maps[0])

(548, 18)
There are 548 raw features...
Building tree structure...
Found tree file...
Populating trees...
There are 304 tree features...


,0,1,2,3,4,5,6,7,8,9,...,489,490,491,492,493,494,495,496,497,498
0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.014875,0.357585,0.046097,0.001915,0.078645,0.498527,0.002356,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.014875,0.100589,0.031811,0.000000,0.225184,0.046097,0.000000,0.001915,0.078645,0.465685,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.014875,0.006186,0.094404,0.031811,0.000000,0.000000,0.000000,0.225184,0.046097,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.014875,0.006186,0.094404,0.000295,0.031517,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.014875,0.006186,0.094404,0.000295,0.031517,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.014875,0.000000,0.006186,0.000000,0.000000,0.000000,0.000000,0.094404,0.000295,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.005155,0.000000,0.001473,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Creating training and test sets
Splitting data into k training and k test sets

In [5]:
#one hot encoding
input = my_maps
target = tf.keras.utils.to_categorical(labels, 2, dtype='int64')
    

# #shuffle dataset
# seed = np.random.randint(100)
# # np.random.seed(seed)
# # np.random.shuffle(input)
# np.random.seed(seed)
# np.random.shuffle(target)

# np.random.seed(seed)
# np.random.shuffle(my_maps)
# np.random.seed(seed)
# np.random.shuffle(raw_x)
# np.random.seed(seed)
# np.random.shuffle(tree_x)
# np.random.seed(seed)
# np.random.shuffle(labels)


#create k training and k test sets
# groups_input = []
# groups_target = []
# k_size = len(input)//k
# start, end = 0, k_size
# for i in range(k):
#     if i == k-1:
#         group_input = input[start:]
#         group_target = target[start:]
#     else:
#         group_input = input[start:end]
#         group_target = target[start:end]
#     start += k_size
#     end += k_size
#     groups_input.append(group_input)
#     groups_target.append(group_target)

# x_train = []
# y_train = []
# x_test = []
# y_test = []
# for i in range(k-1, -1, -1):
#     x_train.append(np.concatenate((groups_input[i-1], groups_input[i-2], groups_input[i-3], groups_input[i-4])))
#     y_train.append(np.concatenate((groups_target[i-1], groups_target[i-2], groups_target[i-3], groups_target[i-4])))

#     x_test.append(groups_input[i])
#     y_test.append(groups_target[i])

# Model

### Training model
Data is log transformed and then a MinMax transformation. Uses CNN that employs skipped residual identity blocks borrowed from the classic ResNet model then a FC Neural Network to make phenotype prediction. Model dimensions printed below.

In [6]:
data_lst = []

# for i in range(k):
#     x_train1 = x_train[i]
#     y_train1 = y_train[i]
#     x_test1 = x_test[i]
#     y_test1 = y_test[i]

#     model = ResNet(height = x_train1.shape[1], width = x_train1.shape[2], channels = 1, classes = 2)
#     model.init_model()

#     model.train(x_train1, y_train1, x_test1, y_test1, dataset, use_weights = False)
#     y_pred = model.predict(x_test1)
#     auc_roc, auc_pr, f1, mcc = model.evaluate(y_test1, y_pred)
#     data_lst.append([auc_roc, auc_pr, f1, mcc])
    
#     #model.model.save_weights(path + "/model_weights.h5")

#     print(y_test1)
#     print(y_pred)
    
# print(model.model.summary())



# n_values = np.max(labels) + 1
# labels_oh = np.eye(n_values)[labels]
# tree_row = my_maps.shape[1]
# tree_col = my_maps.shape[2]

# skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
# fold = 0
# for train_index, test_index in skf.split(my_maps, labels):
#     train_x, test_x = my_maps[train_index,:,:], my_maps[test_index,:,:]
#     train_y, test_y = labels_oh[train_index,:], labels_oh[test_index,:]
        
#     train_x = np.log(train_x + 1)
#     test_x = np.log(test_x + 1)
        
#     c_prob = [0] * len(np.unique(labels))
#     train_weights = []

#     for l in np.unique(labels):
#         a = float(len(labels))
#         b = 2.0 * float((np.sum(labels==l)))
#         c_prob[int(l)] = a/b

#     c_prob = np.array(c_prob).reshape(-1)

#     for l in np.argmax(train_y, 1):
#         train_weights.append(c_prob[int(l)])
#     train_weights = np.array(train_weights)
        
#     scaler = MinMaxScaler().fit(train_x.reshape(-1, tree_row * tree_col))
#     train_x = np.clip(scaler.transform(train_x.reshape(-1, tree_row * tree_col)), 0, 1).reshape(-1, tree_row, tree_col)
#     test_x = np.clip(scaler.transform(test_x.reshape(-1, tree_row * tree_col)), 0, 1).reshape(-1, tree_row, tree_col)

#     train = [train_x, train_y]
#     test = [test_x, test_y]

#     x_train1 = train_x
#     y_train1 = train_y
#     x_test1 = test_x
#     y_test1 = test_y
        
#         y_train1 = train_y
#         y_test1 = test_y
        
#         x_train1 = np.zeros(train_x.shape)
#         x_train1[train_x != 0] = 1
        
#         x_test1 = np.zeros(test_x.shape)
#         x_test1[test_x != 0] = 1
        
        # for i in range(len(train_x)):
        #     for j in range(len(test_x)):
        #         if np.array_equal(train_x[i], test_x[j]):
        #             print('train')
        #             print(train_x[i])
        #             print('test')
        #             print(test_x[j])
        
        
#     model = ResNet(height = train_x.shape[1], width = train_x.shape[2], channels = 1, classes = 2)
#     model.init_model()
#     model.train(train_x, train_y, test_x, y_test1, dataset, use_weights = False)
#     y_pred = model.predict(test_x)
#     auc_roc, auc_pr, f1, mcc = model.evaluate(test_y, y_pred)
#     data_lst.append([auc_roc, auc_pr, f1, mcc])
#     #model.model.save_weights(path + "/model_weights.h5")
#     print(test_y)
#     print(y_pred)
#     print(model.model.summary())
    
#     fold += 1
#run += 1

model = ResNet(height = input.shape[1], width = input.shape[2], channels = 1, classes = 2)
model.init_model()
model.train(input, target, input, target, dataset, use_weights = False)
y_pred = model.predict(input)
auc_roc, auc_pr, f1, mcc = model.evaluate(target, y_pred)
data_lst.append([auc_roc, auc_pr, f1, mcc])
#model.model.save_weights(path + "/model_weights.h5")
print(target)
print(y_pred)
print(model.model.summary())
            
        
        

Metal device set to: Apple M1 Pro
Epoch 1/150
3/3 [==============================] - 2s 255ms/step - loss: 2.9957 - accuracy: 0.6111 - val_loss: 2.7141 - val_accuracy: 0.5000
Epoch 2/150
3/3 [==============================] - 0s 133ms/step - loss: 2.5656 - accuracy: 0.5556 - val_loss: 2.3309 - val_accuracy: 0.5000
Epoch 3/150
3/3 [==============================] - 0s 105ms/step - loss: 2.2333 - accuracy: 0.5556 - val_loss: 2.0212 - val_accuracy: 0.5000
Epoch 4/150
3/3 [==============================] - 0s 100ms/step - loss: 1.9024 - accuracy: 0.6111 - val_loss: 1.7764 - val_accuracy: 0.5000
Epoch 5/150
3/3 [==============================] - 0s 102ms/step - loss: 1.6875 - accuracy: 0.6111 - val_loss: 1.5866 - val_accuracy: 0.5000
Epoch 6/150
3/3 [==============================] - 0s 96ms/step - loss: 1.4917 - accuracy: 0.7222 - val_loss: 1.4427 - val_accuracy: 0.5000
Epoch 7/150
3/3 [==============================] - 0s 99ms/step - loss: 1.3414 - accuracy: 0.7222 - val_loss: 1.3360 - va

# Displaying Accuracy Metrics and Saving Metrics

Option to save results of all k folds and weights of last model into same directy as data.

In [7]:
col = [str(i) for i in range(1,k+1)]    
results_df = pd.DataFrame(data_lst, columns = ['auc(roc)', 'auc(pr)', 'f1', 'mcc'])
results_df = results_df.transpose()
results_df.columns = col

#results_df.to_csv(path + "/results.csv")
results_df

ValueError: Length mismatch: Expected axis has 1 elements, new values have 5 elements

# Saving Model Weights

Option to save model weights of last model in k fold into same directy as data.

In [8]:
model.model.save_weights(path + "/model_weights.h5")

## 